# Building Conductance-based Neuron Models

There are basically two types of neuron models - conductance-based models and simplified models. In conductance-based models, a single neuron can be regarded as a electric circuit, where the membrane is a capacitor, ion channels are conductors, and ion gradients are batteries. The neuronal activity is captured by the current flow through those ion channels. Sometimes there is an external input to this neuron, which can also be included in the equivalent circuit (see the figure below which shows potassium channels, sodium channels and leaky channels).

<img src="../_static/conductance_model_diagram.png" width="400 px">

On the other hand, simplified models do not care about the physiological features of neurons but mainly focus on how to reproduce the exact spike timing. Therefore, they are more simplified and maybe not biologically explicable.

BrainPy provides a large volume of predefined neuron models including conductance-based and simplified models for ease of use. In this section, we will only talk about how to build conductance-based models by ion channels. Users please refer to [Customizing Your Neuron Models](customize_neuron_models.ipynb) for more information.

## Building an ion channel

As we have known, ion channels are crucial for conductance-based neuron models. So how do we model an ion channel? Let's take a look at the potassium channel for instance.

<img src="../_static/potassium_channel_equivalent_circuit.png" width="500 px">

The diagram above shows how a potassium channel is changed to a electric circuit. When a current flows through the channel, we have the differential equation:

$$
c_\mathrm{M} \frac{\mathrm{d}V_\mathrm{M}}{\mathrm{d}t} = \frac{E_\mathrm{K} - V_\mathrm{M}}{r_\mathrm{K}},
$$

in which we define from inside to outside as the positive direction.